In [14]:
import pandas as pd
import pyarrow

anime = pd.read_csv("anime.csv")
rating = pd.read_csv("rating.csv")

anime.to_parquet('anime.parquet')
rating.to_parquet('rating.parquet')



In [15]:
rating.shape

(7813737, 3)

In [16]:
anime.isnull().sum()

Unnamed: 0      0
anime_id        0
name            0
genre          62
type           25
episodes        0
rating        230
members         0
dtype: int64

In [17]:
rating.isnull().sum()

user_id     0
anime_id    0
rating      0
dtype: int64

In [18]:
anime.duplicated().sum()

0

In [19]:
rating.duplicated().sum()

1

In [20]:
merged = rating.merge(anime,on='anime_id')

In [21]:
merged

,user_id,anime_id,rating_x,Unnamed: 0,name,genre,type,episodes,rating_y,members
0,1,20,-1,841,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
1,3,20,8,841,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
2,5,20,6,841,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
3,6,20,-1,841,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
4,10,20,-1,841,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
...,...,...,...,...,...,...,...,...,...,...
7813722,65682,30450,8,8493,Dr. Slump: Hoyoyo! Arale no Himitsu Dai Koukai...,"Comedy, Sci-Fi, Shounen",Special,1,6.17,248
7813723,69497,33484,10,10256,Shiroi Zou,"Action, Historical, Kids",Movie,1,4.71,45
7813724,70463,29481,-1,9097,Kakinoki Mokkii,"Fantasy, Kids",Special,1,4.33,61
7813725,72404,34412,-1,8777,Hashiri Hajimeta bakari no Kimi ni,Music,Music,1,6.76,239


In [22]:
numr = merged.groupby('anime_id').count()['rating_x'].reset_index()
numr.rename(columns={'rating_x':'num_ratings'},inplace=True)
numr

,anime_id,num_ratings
0,1,15509
1,5,6927
2,6,11077
3,7,2629
4,8,413
...,...,...
11192,34367,5
11193,34412,1
11194,34475,4
11195,34476,1


In [23]:
numr.head(5000)

,anime_id,num_ratings
0,1,15509
1,5,6927
2,6,11077
3,7,2629
4,8,413
...,...,...
4995,7430,73
4996,7435,13
4997,7436,14
4998,7445,9


In [24]:
x = merged.groupby('user_id').count()['rating_x'] > 150

In [25]:
cultured = x[x].index

filtered = merged[merged['user_id'].isin(cultured)]

y = filtered.groupby('anime_id').count()['rating_x']>=150
eanime = y[y].index

final = filtered[filtered['anime_id'].isin(eanime)]

lfinal = final.pivot_table(index='anime_id',columns='user_id',values='rating_x')

lfinal.fillna(0,inplace=True)

In [26]:
lfinal.to_parquet('lfinal.parquet')



In [27]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_scores = cosine_similarity(lfinal)

similarity_scores.shape

(3691, 3691)

In [28]:
import numpy as np
similarity_scores
similarity_scores = pd.DataFrame(similarity_scores)
similarity_scores.to_parquet('similarity_scores.parquet')
similarity_scores = similarity_scores.values


In [29]:

lfinal.index

Index([    1,     5,     6,     7,     8,    15,    16,    17,    18,    19,
       ...
       33201, 33222, 33338, 33421, 33524, 33558, 33569, 33964, 34103, 34240],
      dtype='int64', name='anime_id', length=3691)

In [30]:
def get_recommendation(anime_id):
    index = np.where(lfinal.index==anime_id)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:50]
    data = []
    for i in similar_items:
        temp_df = anime[anime['anime_id'] == lfinal.index[i[0]]]
        anime_id = temp_df.drop_duplicates('anime_id')['anime_id'].values[0]
        data.append(anime_id)
    return data[:27]

In [32]:
get_recommendation(20)

[1535,
 1575,
 11757,
 269,
 121,
 5114,
 16498,
 9919,
 2904,
 8074,
 226,
 3588,
 6547,
 442,
 813,
 4224,
 2472,
 6702,
 10620,
 936,
 6880,
 4437,
 223,
 2167,
 356,
 6746,
 2144]

In [33]:
get_recommendation(1735)

IndexError: index 0 is out of bounds for axis 0 with size 0